# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [200]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('/Users/namraschazil/scaling_to_production/05_src/data/fire/forestfires.csv', header = None, names = columns))


In [201]:
fires_dt

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
1,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0,0
2,7,4,oct,tue,90.6,35.4,669.1,6.7,18,33,0.9,0,0
3,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0,0
4,8,6,mar,fri,91.7,33.3,77.5,9,8.3,97,4,0.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0,6.44
514,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0,54.29
515,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0,11.16
516,1,4,aug,sat,94.4,146,614.7,11.3,25.6,42,4,0,0


# Get X and Y

Create the features data frame and target data.

In [202]:
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor

In [203]:
X = fires_dt.drop(columns= 'area')
y = fires_dt['area']
X_train, X_test, Y_train, Y_test = train_test_split (X, Y, test_size = 0.3, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [204]:
# preproc1
num_cols = ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area']
cat_cols = [ 'month', 'day']

pipe_num_simple = Pipeline([
    ('standardizer', RobustScaler(with_centering= True, with_scaling=True, quantile_range=(25.0,75.0), copy=True, unit_variance=False)),
])
pipe_cat_simple = Pipeline([
    ('onehot', OneHotEncoder(categories='auto', drop=None, sparse_output=True, handle_unknown='error',min_frequency=None, max_categories=None, feature_name_combiner='concat'))
])
preproc1= ColumnTransformer([
    ('num_transforms', pipe_num_simple, num_cols),
    ('cat_transforms', pipe_cat_simple, cat_cols)
], 
)
preproc1


ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('standardizer',
                                                  RobustScaler())]),
                                 ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                  'isi', 'temp', 'rh', 'wind', 'rain',
                                  'area']),
                                ('cat_transforms',
                                 Pipeline(steps=[('onehot', OneHotEncoder())]),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [205]:
# preproc2
num_cols = ['coord_x', 'coord_y', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area']
cat_cols = [ 'month', 'day']

num_cols_transform = ['ffmc']

pipe_num_yj = Pipeline([
    ('power_transform', PowerTransformer(method='yeo-johnson')),
])

pipe_num_simple = Pipeline([
('standardizer', RobustScaler(with_centering= True, with_scaling=True, quantile_range=(25.0,75.0), copy=True, unit_variance=False)),
])
 
pipe_cat_simple = Pipeline([
    ('onehot', OneHotEncoder(categories='auto', drop=None, sparse_output=True, handle_unknown='error',min_frequency=None, max_categories=None, feature_name_combiner='concat'))
])

preproc2= ColumnTransformer([
    ('num_transform_simple', pipe_num_simple, num_cols),
    ('num_transform_yj', pipe_num_yj, num_cols_transform),
    ('cat_transform', pipe_cat_simple,cat_cols )
])


preproc2

ColumnTransformer(transformers=[('num_transform_simple',
                                 Pipeline(steps=[('standardizer',
                                                  RobustScaler())]),
                                 ['coord_x', 'coord_y', 'dmc', 'dc', 'isi',
                                  'temp', 'rh', 'wind', 'rain', 'area']),
                                ('num_transform_yj',
                                 Pipeline(steps=[('power_transform',
                                                  PowerTransformer())]),
                                 ['ffmc']),
                                ('cat_transform',
                                 Pipeline(steps=[('onehot', OneHotEncoder())]),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [206]:

# Pipeline A = preproc1 + baseline


regressor = LogisticRegression()
pipeline_A= Pipeline([
    ('preproc1', preproc1),
    ('regressor', regressor )
])
pipeline_A


Pipeline(steps=[('preproc1',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain',
                                                   'area']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['month', 'day'])])),
                ('regressor', LogisticRegression())])

In [207]:
# Pipeline B = preproc2 + baseline

regressor = LogisticRegression()
pipeline_B= Pipeline([
    ('preproc2', preproc2),
    ('regressor', regressor )
])
pipeline_B

Pipeline(steps=[('preproc2',
                 ColumnTransformer(transformers=[('num_transform_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'dmc',
                                                   'dc', 'isi', 'temp', 'rh',
                                                   'wind', 'rain', 'area']),
                                                 ('num_transform_yj',
                                                  Pipeline(steps=[('power_transform',
                                                                   PowerTransformer())]),
                                                  ['ffmc']),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['month', 'day'])])),
                ('regressor', LogisticRegression())])

In [208]:
# Pipeline C = preproc1 + advanced model

advanced_regressor = RandomForestRegressor()
Pipeline_C= Pipeline([
    ('preproc1', preproc1),
    ('regressor', advanced_regressor )
])
Pipeline_C

Pipeline(steps=[('preproc1',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain',
                                                   'area']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor())])

In [209]:
def evaluate_model(model_pipeline, X, y):
    cv_scores = cross_val_score(model_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-cv_scores)
    r2_scores = cross_val_score(model_pipeline, X, y, cv=5, scoring='r2')

In [210]:
# Pipeline D = preproc2 + advanced model
advanced_regressor = RandomForestRegressor()
Pipeline_D= Pipeline([
    ('preproc1', preproc2),
    ('regressor', advanced_regressor )
])
Pipeline_D

Pipeline(steps=[('preproc1',
                 ColumnTransformer(transformers=[('num_transform_simple',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'dmc',
                                                   'dc', 'isi', 'temp', 'rh',
                                                   'wind', 'rain', 'area']),
                                                 ('num_transform_yj',
                                                  Pipeline(steps=[('power_transform',
                                                                   PowerTransformer())]),
                                                  ['ffmc']),
                                                 ('cat_transform',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['month', 'day'])])),
                ('regressor', RandomForestRegressor())])

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [211]:
pipeline_A.get_params()

{'memory': None,
 'steps': [('preproc1',
   ColumnTransformer(transformers=[('num_transforms',
                                    Pipeline(steps=[('standardizer',
                                                     RobustScaler())]),
                                    ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                                     'isi', 'temp', 'rh', 'wind', 'rain',
                                     'area']),
                                   ('cat_transforms',
                                    Pipeline(steps=[('onehot', OneHotEncoder())]),
                                    ['month', 'day'])])),
  ('regressor', LogisticRegression())],
 'verbose': False,
 'preproc1': ColumnTransformer(transformers=[('num_transforms',
                                  Pipeline(steps=[('standardizer',
                                                   RobustScaler())]),
                                  ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc',
                              

In [212]:
X = fires_dt.drop(columns = 'area')
Y = fires_dt['area']

#scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)


In [213]:
param_grid_A = {
    'clf__C': [0.01, 0.5, 1.0],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear'],
}
param_grid_A

{'clf__C': [0.01, 0.5, 1.0],
 'clf__penalty': ['l1', 'l2'],
 'clf__solver': ['liblinear']}

In [214]:
grid_cv_A = GridSearchCV(
    estimator=pipeline_A, 
    param_grid=param_grid_A, 
    cv = 5,
    scoring = "neg_mean_squared_error")
grid_cv_A.fit(X, Y)

The least populated class in y has only 1 members, which is less than n_splits=5.


ValueError: Invalid parameter 'clf' for estimator Pipeline(steps=[('preproc1',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('standardizer',
                                                                   RobustScaler())]),
                                                  ['coord_x', 'coord_y', 'ffmc',
                                                   'dmc', 'dc', 'isi', 'temp',
                                                   'rh', 'wind', 'rain',
                                                   'area']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  ['month', 'day'])])),
                ('regressor', LogisticRegression())]). Valid parameters are: ['memory'

In [ ]:
pipeline_B.get_params()
param_grid_B = {
     'memory':[None], 
     'steps':[('standardizer', RobustScaler())],
     'verbose': [0]
}
grid_cv_B = GridSearchCV(
    estimator=pipeline_B, 
    param_grid=param_grid_B, 
    scoring = "neg_mean_squared_error", 
    cv = 5)
grid_cv_B.fit(X_train, Y_train)
grid_cv_B

The least populated class in y has only 1 members, which is less than n_splits=5.


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 95, in validate_parameter_constraints
 

In [ ]:
Pipeline_C.get_params()
param_grid_C = {
    'regressor__verbose': [0,1,2]
}
grid_cv = GridSearchCV(
    estimator=Pipeline_C, 
    param_grid=param_grid_C, 
    scoring = "neg_mean_squared_error", 
    cv = 5)
grid_cv.fit(X_train, Y_train)
grid_cv

ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'area'

The above exception was the direct cause of the following exception:

Traceback (most re

In [ ]:
Pipeline_D.get_params()
param_grid_D = {
    'regressor__verbose': [0,1,2]
}
grid_cv = GridSearchCV(
    estimator=Pipeline_D, 
    param_grid=param_grid_D, 
    scoring = "neg_mean_squared_error", 
    cv = 5)
grid_cv.fit(X_train, Y_train)
grid_cv

ValueError: 
All the 15 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/pandas/core/indexes/base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'area'

The above exception was the direct cause of the following exception:

Traceback (most re

# Evaluate

+ Which model has the best performance?

In [ ]:
grid_cv_A.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [ ]:
print("Baseline Regressor - Best parameters found:")
print(grid_cv_A.best_params_)
print("Baseline Regressor - Best MSE found:")
print(-grid_cv_A.best_score_)

Baseline Regressor - Best parameters found:


AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [215]:
import shap

In [216]:
best_model = grid_cv_B.best_estimator_
explainer = shap.explainers(best_model.named_steps['regressor'])
observation_idx = 0
observation = X_test[observation_idx]
shap_values = explainer(observation)
shap.summary_plot(shap_values, features=observation, feature_names=X_test.columns)

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [217]:
shap_values_train = explainer(X_train)
shap.summary_plot(shap_values_train, features=X_train, feature_names=X_train.columns)

NameError: name 'explainer' is not defined

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.